In [ ]:
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

class Survey:
    def __init__(self, webdriver_path, form_url, target_response):
        self.webdriver_path = webdriver_path
        self.form_url = form_url
        self.target_response = target_response
        self.complete = 0

        # Initialize the browser
        service = Service(webdriver_path)
        self.driver = webdriver.Chrome(service=service)
        self.driver.maximize_window()

    @staticmethod
    def random_text():
        """Generate random text for form fields."""
        return random.choice(['Yes', 'No', 'Probably', 'Maybe', 'Not sure'])

    def fill_form(self):
        """Fill out the form fields automatically based on their type."""
        try:

            # o1 = self.driver.find_elements(By.XPATH, '//div[@role="radio"]')
            # l1 = [option.get_attribute('aria-label') for option in o1 if option.get_attribute('aria-label')]
            #
            # o2 = self.driver.find_elements(By.XPATH, '//div[@role="checkbox"]')
            # l2 = [option.get_attribute('aria-label') for option in o2 if option.get_attribute('aria-label')]
            #
            # o3 = self.driver.find_elements(By.XPATH, '//div[@role="option" and @aria-selected="false"]')
            # l3 = [option.get_attribute('data-value') for option in o3 if option.get_attribute('data-value')]

            # Handle radio group questions
            radiogroups = self.driver.find_elements(By.XPATH, '//div[@role="radiogroup"]')
            for radiogroup in radiogroups:
                options = radiogroup.find_elements(By.XPATH, './/div[@role="radio"]')
                #print(1)
                #print(options.get_attribute('aria-label'))
                if options:
                    random.choice(options).click()

            # Handle checkbox questions
            checkboxes = self.driver.find_elements(By.XPATH, '//div[@role="listitem"]')
            for checkbox_group in checkboxes:
                checkboxes = checkbox_group.find_elements(By.XPATH, './/div[@role="checkbox"]')
                #print(2)
                #print(checkboxes.get_attribute('aria-label'))
                if checkboxes:
                    random.choice(checkboxes).click()

            # Handle short text fields
            short_text_fields = self.driver.find_elements(By.XPATH, '//input[@type="text"]')
            for short_field in short_text_fields:
                short_field.send_keys(self.random_text())

            # Handle long text fields
            long_text_fields = self.driver.find_elements(By.XPATH, '//textarea')
            for long_field in long_text_fields:
                long_field.send_keys(self.random_text())

            # Handle dropdown questions
            drop_groups = self.driver.find_elements(By.XPATH, '//div[@role="presentation"]')
            if drop_groups:
                for drop_group in drop_groups:
                    try:
                        drop_group.click()
                        time.sleep(0.3)
                        select_option = drop_group.find_elements(By.XPATH, './/div[@role="option" and @aria-selected="false"]')
                        #print(3)
                        #print(select_option.get_attribute('data-value'))
                        if select_option and isinstance(select_option, list):
                            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(select_option))
                            random.choice(select_option).click()
                    except Exception as e:
                        print(f"Error: {e}")

            time.sleep(0.2)  # Delay between processing each question

        except Exception as e:
            print(f"Error while filling form: {e}")

    def click_next(self):
        """Click the 'Next' button if available."""
        try:
            next_button = self.driver.find_element(By.XPATH, '//span[contains(text(), "Tiếp")]')
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(next_button))
            next_button.click()
            self.driver.execute_script("document.body.style.zoom='50%'")
            time.sleep(1)
            return True
        except Exception as e:
            print(f"Error while clicking next: {e}")
            return False

    def submit_form(self):
        """Click the 'Submit' button to submit the form."""
        try:
            submit_button = self.driver.find_element(By.XPATH, '//span[contains(text(), "Gửi")]')
            WebDriverWait(self.driver, 10).until(EC.element_to_be_clickable(submit_button))
            submit_button.click()
            self.driver.execute_script("document.body.style.zoom='50%'")
            self.complete += 1
            time.sleep(1)  # Wait for page reload after submission
        except Exception as e:
            print(f"Error while submitting form: {e}")

    def start(self):
        """Start the survey automation."""
        try:
            while self.complete < self.target_response:
                self.driver.get(self.form_url)
                self.driver.execute_script("document.body.style.zoom='50%'")
                time.sleep(1)  # Wait for the page to load

                # Lặp để điền form và bấm 'Next' đến khi không còn nút 'Next'
                while True:
                    self.fill_form()  # Gọi hàm điền form
                    if self.click_next():  # Nếu bấm được nút 'Next'
                        time.sleep(1)  # Đợi trang kế tiếp tải
                    else:
                        # Nếu không có nút 'Next', bấm 'Submit' và thoát vòng lặp
                        self.submit_form()
                        break

                print(f"Form submitted. Total responses submitted: {self.complete}/{self.target_response}")

            print(f"Task complete. Total responses submitted: {self.complete}/{self.target_response}")
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            try:
                self.driver.quit()
            except Exception as quit_error:
                print(f"Browser already closed. Clean exit. ({quit_error})")


In [ ]:
# Khởi tạo và chạy Survey
webdriver_path1 = "data/chromedriver.exe"  # Đường dẫn đến ChromeDriver
form_url1 = "https://forms.gle/n3SNAgbZ2Wg82Q219"  # URL Google Form
target_response1 = 1  # Số lượng phản hồi mục tiêu

survey = Survey(webdriver_path1, form_url1, target_response1)
survey.start()
